In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
import os
token = os.getenv('GITHUB_TOKEN', '')
import lex, lexxml, lexpy, lexjs
lexpy = lexpy.gen(lex)

In [2]:
saving_string = '19000_20000'

In [2]:
with open('../../660_data/py_project_commit_'+saving_string+'.pickle',"rb") as f:
    project_output_dict = pickle.load(f)

In [4]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm
# remove alias

def remove_alias(project_output_dict):
    with tqdm(total=len(list(project_output_dict.keys()))) as pbar:
        for project_url, test_project_commit in project_output_dict.items():
            remove_alias_dict = {}
            for commit, value in test_project_commit.items():
                [author_name, author_email] = project_output_dict[project_url][commit]['author'].split('<')
                # Get Author name, email and email prefix
                project_output_dict[project_url][commit]['author_name'] = author_name
                project_output_dict[project_url][commit]['author_email'] = author_email
                project_output_dict[project_url][commit]['author_prefix'] = author_email.split('@')[0]

                if author_email in remove_alias_dict:
                    remove_alias_dict[author_email].append(author_name)
                else:
                    remove_alias_dict[author_email] = []

            # Find if there is any duplicates in merge author name and email prefix
            for commit, value in test_project_commit.items():
                [author_name, author_email] = project_output_dict[project_url][commit]['author'].split('<')
                author_prefix = author_email.split('@')[0]
                if author_name in remove_alias_dict[author_email] or author_prefix in in remove_alias_dict[author_email]:
                    project_output_dict[project_url][commit]['author_name'] = remove_alias_dict[author_email][0]

            # Find if there is any duplicates in author names but with different email
            remove_alias_dict = {}
            for commit, value in test_project_commit.items():
                [author_name, author_email] = project_output_dict[project_url][commit]['author'].split('<')
                if author_name in remove_alias_dict:
                    remove_alias_dict[author_name].append(author_email)
                else:
                    remove_alias_dict[author_name] = []

            # Merge alias with the same author_email
            for commit, value in test_project_commit.items():
                [author_name, author_email] = project_output_dict[project_url][commit]['author'].split('<')
                if project_output_dict[project_url][commit]['author_name'] in remove_alias_dict[author_name]:
                project_output_dict[project_url][commit]['author_email'] = remove_alias_dict[author_name][0]
        pbar.update(1)
    return project_output_dict

100%|█████████████████████████████████████| 2000/2000 [00:00<00:00, 2682.75it/s]


Extract lexical tokens from github.diff.

In [5]:
def print_py_token(input_string):
    '''
    param:
        input_string: python source codes
    return:
        
    '''
    lexer = lex.Lexer(lexpy, input_string)
    token_list = []
    while (True):
        t = lexer.get_token();
        if (t is None): 
            break;
        token_list.append(lexer.repr_token(t))
    return token_list

def CountFrequency(my_list):
 
    # Creating an empty dictionary
    freq = {}
    for item in my_list:
        if (item in freq):
            freq[item] += 1
        else:
            freq[item] = 1
    return freq
        

def filter_multi_author_py_files(data_dict):
    files_count = {}
    for author, value in data_dict.items():
        for file, tokens in value.items():
            if file in files_count:
                files_count[file] += 1
            else:
                files_count[file] = 1
    result_files = []
    for key, value in files_count.items():
        if value > 1:
            result_files.append(key)
    return result_files


def generate_token_ACE_list(data_dict, project_url, collabrate_files):
    df_record = []
    for author, value in data_dict.items():
        for collabrate_file in collabrate_files:
            if collabrate_file in value:
                for token, count in value[collabrate_file].items():
                    df_record.append([author, collabrate_file, token, count])

    df = pd.DataFrame.from_records(df_record, columns=['author_email', 'file_name', 'token', 'count'])
    token_ACE = []
    for file in df['file_name'].unique():
        temp = df[df['file_name'] == file].groupby('token').agg({'author_email':'nunique'})
        tokens = temp[temp['author_email'] > 1].index.tolist()
        for token in tokens:
            temp_numpy = df[df['token'] == token]['count'].to_numpy()
            total = np.sum(temp_numpy)
            temp_probability = temp_numpy/total
            cross_entropy = -np.sum(temp_probability * np.log(temp_probability))
            token_ACE.append([project_url, token, cross_entropy,file, len(temp_numpy)]) # token, cross_entropy, file, num_author
    return token_ACE

In [8]:
final_result = []
 
with tqdm(total=1000) as pbar:
    for project_url in list(project_output_dict.keys())[:1000]:  
        test_project_commit  = project_output_dict[project_url]
        pbar.update(1)
        data_dict = {}
        url_params = '/'.join(project_url.split('/')[-2:])
        for commit, value in test_project_commit.items():
        #     if len(list(value.keys())) != 0:
            data_dict[value['author_email']] = {}
            query_url = f"https://api.github.com/repos/"+url_params+"/commits/"+commit
            params = {
                "state": "open",
            }
            headers = {'Authorization': f'token {token}', 'Accept': 'application/vnd.github.diff',}
            r = requests.get(query_url, headers=headers, params=params)
            diff_list = list(r.iter_lines())
            python_file_start = False
            token_list = []
            for row in diff_list:
                row = row.decode('utf-8', 'ignore')
        #         if row[:3] == '---' or row[:3] == '+++':
                if row[:3] == '+++':

                    if row[4:][-2:] == 'py':
                        python_file_start = True
                        file = row[4:]
                        token_list = []
                    else:
                        if len(token_list) > 0:
                            data_dict[value['author_email']][file] = CountFrequency(token_list)
                        python_file_start = False
                    continue
    #             if python_file_start and (row[0] == '-' or row[0] == '+'):
                if python_file_start and len(row) > 0 and row[0] == '+':
                    input_string = row[1:].strip(' ')
                    token_list.extend(print_py_token(input_string))

        collabrate_files = filter_multi_author_py_files(data_dict)
        if len(collabrate_files) == 0:
            continue
        token_ACE = generate_token_ACE_list(data_dict, project_url, collabrate_files)
        final_result.extend(token_ACE)

100%|████████████████████████████████████| 1000/1000 [16:05:13<00:00, 57.91s/it]


In [9]:
df = pd.DataFrame.from_records(final_result, columns=['project_url','token', 'cross_entropy', 'file', 'num_author'])
df

,project_url,token,cross_entropy,file,num_author
0,https://github.com/thisisaaronland/go-iiif,"Token(text=""'%s.jpg'"", type=STRING, flags=NEXT...",0.636514,b/vendor/src/github.com/fogleman/primitive/bot...,2
1,https://github.com/thisisaaronland/go-iiif,"Token(text=""'%s.png'"", type=STRING, flags=NEXT...",0.636514,b/vendor/src/github.com/fogleman/primitive/bot...,2
2,https://github.com/thisisaaronland/go-iiif,"Token(text=""''"", type=STRING, flags=NEXT_NO_OP...",0.673012,b/vendor/src/github.com/fogleman/primitive/bot...,2
3,https://github.com/thisisaaronland/go-iiif,"Token(text=""'id'"", type=STRING, flags=NEXT_NO_...",0.562335,b/vendor/src/github.com/fogleman/primitive/bot...,2
4,https://github.com/thisisaaronland/go-iiif,"Token(text=' ', type=WHITESPACE, flags=IGNORE,...",0.683629,b/vendor/src/github.com/fogleman/primitive/bot...,2
...,...,...,...,...,...
20910,https://github.com/clemsonacm/hackpack,"Token(text='type', type=IDENTIFIER, flags=NEXT...",0.693147,b/hackpack/util/convertcppman.py,2
20911,https://github.com/clemsonacm/hackpack,"Token(text='with', type=KEYWORD, flags=NONE, s...",0.693147,b/hackpack/util/convertcppman.py,2
20912,https://github.com/clemsonacm/hackpack,"Token(text='write', type=IDENTIFIER, flags=MEM...",0.693147,b/hackpack/util/convertcppman.py,2
20913,https://github.com/clemsonacm/hackpack,"Token(text='yield', type=KEYWORD, flags=NONE, ...",0.693147,b/hackpack/util/convertcppman.py,2


In [10]:
df.to_pickle('ACE_first_'+saving_string+'.pickle')